In [5]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random
import sqlite3
import tkinter as tk
from tkinter import messagebox
import os
import json
import matplotlib.pyplot as plt

# -------------------- CONFIGURACIÓN DE LA BASE DE DATOS --------------------
folder_path = r"C:\\Users\\carlo\\Documents\\TFG\\juegos\\Datos Juegos"
os.makedirs(folder_path, exist_ok=True)
db_path = os.path.join(folder_path, "semicirculos_game.db")

MEDALLAS = {
    "bronce": r"C:\\Users\\carlo\\Documents\\TFG\\juegos\\Imagenes_Juegos\\Bronce",
    "plata": r"C:\\Users\\carlo\\Documents\\TFG\\juegos\\Imagenes_Juegos\\Plata",
    "oro": r"C:\\Users\\carlo\\Documents\\TFG\\juegos\\Imagenes_Juegos\\Oro"
}

def init_db():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS patients (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT UNIQUE,
        points INTEGER DEFAULT 0,
        rank TEXT DEFAULT 'bronce',
        prestige INTEGER DEFAULT 1
    )
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS sessions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id INTEGER,
        timestamp TEXT,
        collected_count INTEGER,
        failed_count INTEGER,
        mode TEXT,
        total_rounds INTEGER,
        mean_separation REAL,
        separation_array TEXT,
        mean_reaction_time REAL,
        reaction_time_array TEXT,
        score_points INTEGER,
        FOREIGN KEY (patient_id) REFERENCES patients(id)
    )
    """)
    conn.commit()
    return conn, cursor

# -------------------- FUNCIONES AUXILIARES DE GRAFICACIÓN --------------------

def crear_y_guardar_graficas(nombre_paciente, session_id, collected_count, failed_count, mean_reaction_time, separation_array):
    base_dir = os.path.join(folder_path, "graficas", f"{nombre_paciente} interphalangeal joints")
    carpetas = {
        "fallos_vs_aciertos": os.path.join(base_dir, "fallos_vs_aciertos"),
        "tiempo_reaccion": os.path.join(base_dir, "tiempo_reaccion"),
        "distancia_interfalangica_tiempo": os.path.join(base_dir, "distancia_interfalangica_tiempo")
    }
    for ruta in carpetas.values():
        os.makedirs(ruta, exist_ok=True)

    # Fallos vs Aciertos
    plt.figure()
    plt.bar(["Aciertos", "Fallos"], [collected_count, failed_count], color=["green", "red"])
    plt.title("Aciertos vs Fallos")
    plt.savefig(os.path.join(carpetas["fallos_vs_aciertos"], f"sesion_{session_id}.png"))
    plt.close()

    # Tiempo de reacción medio acumulado
    historial_path = os.path.join(carpetas["tiempo_reaccion"], "media_tiempo_reaccion_acumulado.json")
    historial = []
    if os.path.exists(historial_path):
        with open(historial_path, "r") as f:
            historial = json.load(f)
    historial.append(mean_reaction_time)
    with open(historial_path, "w") as f:
        json.dump(historial, f)

    plt.figure()
    plt.plot(range(1, len(historial) + 1), historial, marker='o')
    plt.title("Tiempo de reacción medio por sesión")
    plt.xlabel("Sesión")
    plt.ylabel("Tiempo (s)")
    plt.grid()
    plt.savefig(os.path.join(carpetas["tiempo_reaccion"], "media_tiempo_reaccion_acumulado.png"))
    plt.close()

    # Distancia interfalángica vs tiempo
    tiempos = np.linspace(0, len(separation_array) * 0.1, len(separation_array))
    plt.figure()
    plt.plot(tiempos, separation_array)
    plt.title("Distancia interfalángica vs Tiempo")
    plt.xlabel("Tiempo (s)")
    plt.ylabel("Distancia (px)")
    plt.grid()
    plt.savefig(os.path.join(carpetas["distancia_interfalangica_tiempo"], f"sesion_{session_id}.png"))
    plt.close()
# -------------------- FUNCIONES DE GAMIFICACIÓN --------------------
def actualizar_rango(cursor, conn, patient_id, puntos_nuevos):
    cursor.execute("SELECT points, rank, prestige FROM patients WHERE id = ?", (patient_id,))
    fila = cursor.fetchone()
    puntos, rango, prestigio = fila if fila else (0, 'bronce', 1)
    total = puntos + puntos_nuevos

    sube_rango = False
    if total >= 45:
        total = 0
        prestigio += 1
        rango = "bronce"
        sube_rango = True
    elif total >= 30:
        rango = "oro"
    elif total >= 15:
        rango = "plata"
    else:
        rango = "bronce"

    cursor.execute("""
        UPDATE patients SET points = ?, rank = ?, prestige = ? WHERE id = ?
    """, (total, rango, prestigio, patient_id))
    conn.commit()
    return rango, prestigio, sube_rango
def dibujar_mensaje_punto(frame, texto="+1"):
    cv2.putText(frame, texto, (WIDTH - 100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
def verificar_y_mostrar_punto(frame, last_speed, media_velocidad_anterior):
    if last_speed > media_velocidad_anterior:
        dibujar_mensaje_punto(frame)
        return True
    return False


def mostrar_imagen_medalla(rango):
    ruta = MEDALLAS.get(rango.lower())
    if ruta and os.path.isdir(ruta):
        for file in os.listdir(ruta):
            if file.lower().endswith(('.png', '.jpg')):
                path = os.path.join(ruta, file)
                img = cv2.imread(path)
                if img is not None:
                    cv2.imshow("Rango actual", img)
                    cv2.waitKey(3000)
                    cv2.destroyAllWindows()
                    break

# -------------------- BLOQUE FINAL DEL JUEGO --------------------
def finalizar_juego(patient_name, session_id, collected_count, failed_count, mean_reaction, separation_log, reaction_times, conn, cursor, patient_id):
    if not reaction_times:
        reaction_times = []

    mean_velocity_previous = np.mean([1 / t for t in reaction_times if t > 0]) if reaction_times else 0
    puntos_obtenidos = sum(1 for t in reaction_times if t > 0 and (1 / t) > mean_velocity_previous)

    crear_y_guardar_graficas(patient_name, session_id, collected_count, failed_count, mean_reaction, separation_log)

    rango_actual, prestigio_actual, subido = actualizar_rango(cursor, conn, patient_id, puntos_obtenidos)
    if subido:
        messagebox.showinfo("¡Nuevo Rango!", f"¡Has subido de prestigio! Ahora eres {rango_actual.capitalize()} (Prestigio {prestigio_actual})")
    mostrar_imagen_medalla(rango_actual)

    # Mostrar información del jugador al final
    messagebox.showinfo("Estado Final del Jugador", f"Rango actual: {rango_actual.capitalize()}\nPrestigio: {prestigio_actual}\nPuntos actuales: {puntos_obtenidos}/10")

    return puntos_obtenidos
# -------------------- CALIBRACIÓN --------------------
def calibrate_max_distance(cap, calibration_time=5):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    mp_draw = mp.solutions.drawing_utils
    print(f"Calibración: Abre la mano al máximo durante {calibration_time} segundos.")
    start_time = time.time()
    max_dist_found = 0
    while time.time() - start_time < calibration_time:
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)
        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:
                h, w, _ = frame.shape
                lm8 = handLms.landmark[8]
                lm20 = handLms.landmark[20]
                x8, y8 = int(lm8.x * w), int(lm8.y * h)
                x20, y20 = int(lm20.x * w), int(lm20.y * h)
                dist = np.hypot(x20 - x8, y20 - y8)
                if dist > max_dist_found:
                    max_dist_found = dist
                mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)
        cv2.putText(frame, f"Calibrando: {int(time.time()-start_time)}s", (10,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.imshow("Calibración", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
    cv2.destroyWindow("Calibración")
    return int(max(max_dist_found, 40))

# -------------------- MAPEADO LINEAL --------------------
def linear_map(x, a, b, c, d):
    return c + ((x - a) / (b - a)) * (d - c)

# -------------------- CONSTANTES DEL JUEGO --------------------
WIDTH, HEIGHT = 640, 480
semicircle_radius = 40
target_radius = 20
min_distance = 30
max_distance_default = 200
contact_duration_required = 4

# -------------------- FUNCIÓN PARA MOSTRAR LA MEDALLA --------------------
def get_medalla_path(rango):
    base_dir = r"C:\\Users\\carlo\\Documents\\TFG\\juegos\\Imagenes_Juegos"
    ruta_directa = os.path.join(base_dir, f"{rango}.png")
    if os.path.exists(ruta_directa):
        return ruta_directa
    carpeta = os.path.join(base_dir, rango)
    if os.path.exists(carpeta):
        for archivo in os.listdir(carpeta):
            if archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
                return os.path.join(carpeta, archivo)
    return None

def mostrar_imagen_medalla(rango, ventana=None):
    ruta = get_medalla_path(rango)
    if ruta and os.path.exists(ruta):
        img = cv2.imread(ruta)
        if ventana is not None and img is not None:
            h_img, w_img = img.shape[:2]
            h_v, w_v = ventana.shape[:2]
            x_offset = w_v - w_img - 10
            y_offset = h_v - h_img - 10
            if x_offset >= 0 and y_offset >= 0:
                ventana[y_offset:y_offset + h_img, x_offset:x_offset + w_img] = img
        elif img is not None:
            cv2.imshow("Rango actual", img)
            cv2.waitKey(3000)
            if cv2.getWindowProperty("Rango actual", cv2.WND_PROP_VISIBLE) >= 1:
                cv2.destroyWindow("Rango actual")


# -------------------- FUNCIÓN PRINCIPAL DEL JUEGO --------------------
def start_game(params):
    patient_name = params['patient_name']
    mode = params['mode']
    total_rounds = params['total_rounds']
    target_timeout = {'rapido': 5, 'lento': 10}.get(mode, 8)
    punto_ganado = False

    conn, cursor = init_db()
    cursor.execute("SELECT id FROM patients WHERE name = ?", (patient_name,))
    result = cursor.fetchone()
    if result:
        patient_id = result[0]
    else:
        cursor.execute("INSERT INTO patients (name) VALUES (?)", (patient_name,))
        conn.commit()
        patient_id = cursor.lastrowid

     # Mostrar imagen de medalla en grande al iniciar
    path_medalla = get_medalla_path("bronce")
    if path_medalla:
        img = cv2.imread(path_medalla)
        if img is not None:
            img_grande = cv2.resize(img, (600, 600))
            cv2.imshow("Rango actual", img_grande)
            cv2.waitKey(3000)
            try:
                if cv2.getWindowProperty("Rango actual", cv2.WND_PROP_VISIBLE) >= 1:
                    cv2.destroyWindow("Rango actual")
            except:
                pass

    collected_count = 0
    failed_count = 0
    rounds = 0

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    mp_draw = mp.solutions.drawing_utils

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)

    max_distance = calibrate_max_distance(cap, 5)

    target_y = HEIGHT // 2
    last_side = 'right'
    target_x = random.randint(WIDTH // 2, WIDTH - target_radius)
    target_spawn_time = time.time()
    contact_start_time = None

    separation_log = []
    last_log_time = time.time()
    reaction_times = []
    reaction_time_array = []

    while rounds < total_rounds:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)
        displacement = 0
        current_time = time.time()
        current_separation = None

        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:
                h, w, _ = frame.shape
                lm8 = handLms.landmark[8]
                lm20 = handLms.landmark[20]
                x8, y8 = int(lm8.x * w), int(lm8.y * h)
                x20, y20 = int(lm20.x * w), int(lm20.y * h)
                dist = np.hypot(x20 - x8, y20 - y8)
                current_separation = dist
                max_disp = WIDTH - 2 * semicircle_radius
                displacement = int(np.clip(linear_map(dist, min_distance, max_distance, 0, max_disp), 0, max_disp))
                mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)
                cv2.putText(frame, f"Dist: {int(dist)}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

        if current_separation is not None and current_time - last_log_time >= 0.10:
            separation_log.append(current_separation)
            last_log_time = current_time

        semicircle_center = (semicircle_radius + displacement, HEIGHT // 2)
        cv2.ellipse(frame, semicircle_center, (semicircle_radius, semicircle_radius), 0, 0, 180, (0,255,0), -1)
        cv2.circle(frame, (target_x, target_y), target_radius, (0,0,255), -1)

        d = np.hypot(semicircle_center[0] - target_x, semicircle_center[1] - target_y)
        collision = d < semicircle_radius + target_radius

        if collision:
            if contact_start_time is None:
                contact_start_time = current_time
            elif current_time - contact_start_time >= contact_duration_required:
                reaction_time = current_time - target_spawn_time
                reaction_times.append(reaction_time)
                reaction_time_array.append(reaction_time)

                collected_count += 1
                rounds += 1

                last_speed = 1 / reaction_time if reaction_time > 0 else 0
                media_velocidad_anterior = np.mean([1 / t for t in reaction_times if t > 0]) if reaction_times else 0

                if verificar_y_mostrar_punto(frame, last_speed, media_velocidad_anterior):
                    punto_ganado = True
                    punto_momento = current_time

                last_side = 'left' if last_side == 'right' else 'right'
                if last_side == 'left':
                    target_x = random.randint(target_radius, WIDTH // 2 - target_radius)
                else:
                    target_x = random.randint(WIDTH // 2 + target_radius, WIDTH - target_radius)
                target_spawn_time = current_time
                contact_start_time = None
        else:
            contact_start_time = None

        if current_time - target_spawn_time >= target_timeout:
            failed_count += 1
            rounds += 1
            last_side = 'left' if last_side == 'right' else 'right'
            if last_side == 'left':
                target_x = random.randint(target_radius, WIDTH // 2 - target_radius)
            else:
                target_x = random.randint(WIDTH // 2 + target_radius, WIDTH - target_radius)
            target_spawn_time = current_time
            contact_start_time = None

        if punto_ganado and (time.time() - punto_momento) < 1.0:
            dibujar_mensaje_punto(frame)
        else:
            punto_ganado = False

        countdown = max(0, int(target_timeout - (current_time - target_spawn_time)))
        cv2.putText(frame, f"Tiempo: {countdown}s", (WIDTH - 200, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
        cv2.putText(frame, f"Aciertos: {collected_count}", (10, HEIGHT - 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.putText(frame, f"Fallos: {failed_count}", (10, HEIGHT - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        cv2.putText(frame, f"Ronda: {rounds}/{total_rounds}", (WIDTH//2 - 80, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (200,200,200), 2)

        cv2.imshow("Juego Semicirculos", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

    session_id = None
    if collected_count + failed_count > 0:
        session_id = cursor.execute("""
            INSERT INTO sessions (patient_id, timestamp, collected_count, failed_count, mode, total_rounds, 
                                  mean_separation, separation_array, mean_reaction_time, reaction_time_array, score_points)
            VALUES (?, datetime('now'), ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            patient_id,
            collected_count,
            failed_count,
            mode,
            total_rounds,
            np.mean(separation_log) if separation_log else 0,
            json.dumps(separation_log),
            np.mean(reaction_times) if reaction_times else 0,
            json.dumps(reaction_time_array),
            0  # placeholder de puntos_obtenidos, lo actualiza finalizar_juego
        )).lastrowid
        conn.commit()

    finalizar_juego(patient_name, session_id, collected_count, failed_count, 
                    np.mean(reaction_times) if reaction_times else 0,
                    separation_log, reaction_times, conn, cursor, patient_id)
    conn.close()
# -------------------- INTERFAZ DE CONFIGURACIÓN --------------------
def launch_interface():
    root = tk.Tk()
    root.title("Configuración del Juego Semicírculos")
    root.geometry("400x350")
    params = {}

    tk.Label(root, text="Nombre del paciente:", font=("Helvetica", 12)).pack(pady=10)
    entry_name = tk.Entry(root, font=("Helvetica", 12))
    entry_name.pack(pady=5)

    tk.Label(root, text="Modo de juego:", font=("Helvetica", 12)).pack(pady=10)
    mode_var = tk.StringVar(value="normal")
    for text, val in [("Lento (10s)", "lento"), ("Normal (8s)", "normal"), ("Rápido (5s)", "rapido")]:
        tk.Radiobutton(root, text=text, variable=mode_var, value=val, font=("Helvetica", 11)).pack(anchor="w", padx=20)

    tk.Label(root, text="Total de rondas:", font=("Helvetica", 12)).pack(pady=10)
    entry_total = tk.Entry(root, font=("Helvetica", 12))
    entry_total.pack(pady=5)
    entry_total.insert(0, "10")

    def iniciar_juego():
        name = entry_name.get().strip()
        if not name:
            messagebox.showerror("Error", "Debes ingresar un nombre.")
            return
        params['patient_name'] = name
        params['mode'] = mode_var.get()
        try:
            params['total_rounds'] = int(entry_total.get())
        except:
            messagebox.showerror("Error", "El total de rondas debe ser un número entero.")
            return
        root.destroy()
        start_game(params)

    tk.Button(root, text="Iniciar Juego", command=iniciar_juego,
              font=("Helvetica", 14, "bold"), bg="#4CAF50", fg="white", padx=10, pady=5).pack(pady=20)

    root.mainloop()
if __name__ == "__main__":
    launch_interface()


Calibración: Abre la mano al máximo durante 5 segundos.
